# Porcesar los datos de los videos

Este notebook se encarga de transformar los datos crudos provenientes de archivos de video en un formato estructurado y útil para el análisis.

**Objetivo**
*  Leer y procesar los archivos crudos para crear un DataFrame que contenga información esencial sobre cada grabación, incluyendo detalles sobre el paciente, el número de repetición, el ID del movimiento, la precisión del gesto (si es correcto o no), y la posición de los keypoints.
* Crear un segundo DataFrame que contenga los ángulos calculados entre algunos de los keypoints, lo que ayudará a entender mejor la postura y el movimiento durante la ejecución del gesto.

**Entrada**
* Archivos de datos de videos dentro del directorio ``SkeletonData/RawData``, que contienen información sobre el paciente, las repeticiones, los IDs de movimiento, y las posiciones de keypoints.

**Salida**
* ``Resultados/raw_pacientes.csv``: contiene información detallada sobre cada grabación, incluyendo 
* ``Resulados/angles.csv``: incluye los ángulos calculados entre keypoints, facilitando el análisis de la postura y el movimiento durante el gesto.

**Índice**
1. [Procesar Raw Data Folder](#1-procesar-raw-data-folder)\
    1.1 [Datos de los nombres de los archivos](#11-datos-de-los-nombres-de-los-archivos)\
    1.2 [Datos de los archivos](#12-datos-de-los-archivos)
2. [Calcular los ángulos](#2-calcular-los-ángulos)


In [2]:
# importar librerias necesarias
import pandas as pd # para manejar dataframes
import os # para interactuar con el sistema operativo
import numpy as np

*****
## 1. Procesar Raw Data Folder

El conjunto de datos incluye a 30 sujetos, de los cuales 14 son sanos y 16 son pacientes. Cada participante realiza 9 tipos de ejercicios, con 6 repeticiones de cada uno.

La carpeta contiene 2598 archivos de texto con información de los videos, donde cada archivo corresponde a un sujeto realizando una repetición de un gesto. Hay información tanto en el contenido del archivo como en el propio nombre del archivo.

* En el nombre del archivo se encuentran los identificadores del sujeto, del día, del gesto, el número de repetición, si es correcto y la posición del sujeto durante el ejercicio.

* Dentro de cada archivo hay información sobre las coordenadas de 25 puntos del cuerpo captados por un sensor que graba a 30 fps. Por lo tanto, por cada segundo de video, se generan 30 líneas en el archivo de texto. Cada línea sigue el siguiente formato: marca de tiempo, XX, XX, seguido de 25 pares de (NombreArticulación, EstadoSeguimiento, coordenada 3D X, coordenada 3D Y, coordenada 3D Z, coordenada 2D X, coordenada 2D Y).


### 1.1 Datos de los nombres de los archivos

Los nombre de los archivos siguen el siguiente formato: *SubjectID_DateID_GestureLabel_RepetitionNumber_CorrectLabel_Position.txt*

* *SubjectID*: id uniquely identifying the person performing the exercise
* *DateID*: id identifying the session in which the person was recorded
* *GestureLabel*: Label identifying the gesture; possible values are from 0 to 8
* *RepetitionNumber*: Each gesture was repeated several times and this shows the repetition number
* *CorrectLabel*: A value of 1 represents a gesture labeled as being correctly executed, while a value of 2 is for a gesture labeled as incorrect
* *Position*: Some of the persons performed the gestures sitting on a chair or wheelchair, while others standing

In [10]:
# Función para guardar en el dataframe los datos que aparecen en los nombre de los archivos
def leer_nombre_archivo(archivo:str) -> list[str]:
    """
    Extrae datos específicos del nombre de un archivo de texto.

    Parámetros
    ----------
    archivo : str
        Nombre del archivo, con el formato 'SubjectID_DateID_GestureLabel_RepetitionNumber_CorrectLabel_Position.txt'.

    Return
    -------
    campos : list[str]
        Lista de cadenas de texto que contiene los datos extraídos del nombre del archivo:
        [SubjectID, DateID, GestureLabel, RepetitionNumber, CorrectLabel, Position].
    """
    archivo = archivo.split('.')[0] # quita la extension txt
    campos = archivo.split('_') # separa los campos por _
    return campos

### 1.2 Datos de los archivos
Each raw data file contains per line: timestamp, XX, XX, followed by a 25 pairs of (JointName, TrackedStatus, 3d coordinate X, 3d coordinate Y, 3d coordinate Z, 2d coordinate X, 2d coordinate Y)

In [11]:
# Función para extraer la información de dentro de los archivos
def leer_datos_archivo(directorio:str, columnas:list[str]) -> pd.Dataframe:
    """
    Compila la información de los archivos en un directorio y los guarda en un DataFrame.

    Parámetros
    ----------
    directorio : str
        Nombre del directorio donde se encuentran los archivos.
    columnas : list[str]
        Lista con los nombres de las columnas para el DataFrame de salida.

    Return
    -------
    pd.DataFrame
        DataFrame con todos los datos recopilados de los archivos.
    """
    # Crea una lista con los nombres de los archivos en el directorio
    file_list = os.listdir(directorio)

    # lista para almacenar los datos extraídos
    list_data = []

    # Itera sobre cada archivo
    for file_name in file_list:
         # Extrae los datos del nombre del archivo
        campos = leer_nombre_archivo(file_name)

        with open(os.path.join(directorio, file_name), 'r') as file:
            for line in file:
                 # Divide la línea por comas y extrae la información deseada
                 # omitiendo timestamp y otros datos innecesarios
                line_data = line.strip().split(',')[3:]
                # Quita los paréntesis
                cleaned_data = [item.replace('(', '').replace(')', '') for item in line_data] 
                # por cada linea de los archivos necesitamos bloques de 7 valores
                for i in range(0, len(cleaned_data), 7): 
                    list_data.append(campos + cleaned_data[i:i + 7])
    df = pd.DataFrame(list_data, columns=columnas)
    return df


In [12]:
# directorio donde se encuentran los datos
directory = '../dataset/SkeletonData/RawData' 

columnas = ['SubjectID', 'DateID', 'GestureLabel', 'RepetitionNumber', 'CorrectLabel', 'Position',
            'JointName', 'TrackedStatus', '3D_X', '3D_Y', '3D_Z', '2D_X', '2D_Y']

# Extraer la informacion y almacenarla en un DataFrame
df_data = leer_datos_archivo(directory, columnas)

In [13]:
# Eliminar columnas innceserarias
df_data.drop(['TrackedStatus', 'DateID', '2D_X', '2D_Y'], axis=1, inplace=True)
df_data

,SubjectID,GestureLabel,RepetitionNumber,CorrectLabel,Position,JointName,3D_X,3D_Y,3D_Z
0,101,0,1,1,stand,SpineBase,-0.1028086,0.06965441,2.464606
1,101,0,1,1,stand,SpineMid,-0.1026228,0.3837799,2.438919
2,101,0,1,1,stand,Neck,-0.1025293,0.6877351,2.40196
3,101,0,1,1,stand,Head,-0.1190992,0.8358598,2.373549
4,101,0,1,1,stand,ShoulderLeft,-0.2826451,0.5525576,2.38421
...,...,...,...,...,...,...,...,...,...
5707520,307,8,9,1,stand,SpineShoulder,-0.05799517,0.5291457,2.422904
5707521,307,8,9,1,stand,HandTipLeft,-0.302538,-0.1131345,2.284269
5707522,307,8,9,1,stand,ThumbLeft,-0.2783904,-0.06298634,2.269769
5707523,307,8,9,1,stand,HandTipRight,0.1525867,-0.136378,2.45287


In [14]:
# guardar los datos en formato csv
df_data.to_csv('../Resultados/raw_pacientes.csv', index=False)

******
## 2. Calcular los ángulos

<div style="text-align: center;">
<img src="../images/gestures.png" width="500"/>
</div>

In [15]:
# Función para calcular los ángulos
def calculate_angle(df: pd.DataFrame, joint_a: str, joint_b: str, joint_c: str) -> float:
    """
    Calcula el ángulo entre dos vectores definidos por tres puntos (keypoints) en un DataFrame.

    Parámetros
    -------
    df : pd.DataFrame
        DataFrame que contiene los datos de los keypoints, con una columna 'JointName' y columnas de coordenadas '3D_X', '3D_Y', '3D_Z'.
    joint_a : str
        Nombre del primer keypoint.
    joint_b : str
        Nombre del segundo keypoint (punto de conexión entre los dos segmentos).
    joint_c: str
        Nombre del tercer keypoint.

    Return
    -------
    float : El ángulo en grados entre los dos vectores formados por los tres keypoints.

    """
    # Extraer posiciones de los keypoints
    positions = df.set_index('JointName')[['3D_X', '3D_Y', '3D_Z']].loc[[joint_a, joint_b, joint_c]]

    # # Convertir las posiciones a tipo numérico
    positions = positions.apply(pd.to_numeric)

    # Vector u (joint_a to joint_b) y Vector v (joint_b to joint_c)
    u = np.array([positions.iloc[1, 0] - positions.iloc[0, 0], positions.iloc[1, 1] - positions.iloc[0, 1], positions.iloc[1, 2] - positions.iloc[0, 2]])
    v = np.array([positions.iloc[2, 0] - positions.iloc[1, 0], positions.iloc[2, 1] - positions.iloc[1, 1], positions.iloc[2, 2] - positions.iloc[1, 2]])
  
    # Producto vectorial y modulo de los vectores
    producto_vectorial = np.dot(u, v)
    modulo_u = np.linalg.norm(u)
    modulo_v = np.linalg.norm(v)

    # Comprobar que ninguno de los módulos sea 0
    if (modulo_u * modulo_v) == 0:
        return 0

    # Caclulo del coseno
    cos_angle = producto_vectorial / (modulo_u * modulo_v)

    # Calcular el angulo
    angle = np.arccos(cos_angle) * 180.0 / np.pi

    # Asugurar que el ángulo esté entre los 180 grados
    if angle > 180.0:
        angle = 360 - angle

    return angle

In [16]:
# Aplica la funcióin para caluclar los ángulos a los datos en crudo
# Agrupar el DataFrame por cada 25 filas
groups = [df_data.iloc[i:i+25] for i in range(0, len(df_data), 25)]

# Iterar sobre cada grupo de 25 filas
angles = []
for group in groups:
    # Calcular los ángulos para los keypoints específicos    
    elbow_angle_left = calculate_angle(group, 'ShoulderLeft', 'ElbowLeft', 'WristLeft')
    elbow_angle_right = calculate_angle(group, 'ShoulderRight', 'ElbowRight', 'WristRight')
    left_arm_angle = calculate_angle(group, 'HipLeft', 'ShoulderLeft', 'ElbowLeft')
    right_arm_angle = calculate_angle(group, 'HipRight', 'ShoulderRight', 'ElbowRight')
    arms_together_angle = calculate_angle(group, 'SpineBase', 'SpineShoulder', 'WristLeft')

    elbow_angle_left = calculate_angle(group, 'ShoulderLeft', 'ElbowLeft', 'WristLeft')
    wrist_angle_left = calculate_angle(group, 'ElbowLeft', 'WristLeft', 'HandLeft')
    shoulder_angle_left = calculate_angle(group, 'ShoulderLeft', 'SpineShoulder', 'ElbowLeft')

    elbow_angle_right = calculate_angle(group, 'ShoulderRight', 'ElbowRight', 'WristRight')
    wrist_angle_right = calculate_angle(group, 'ElbowRight', 'WristRight', 'HandRight')
    shoulder_angle_right = calculate_angle(group, 'ShoulderRight', 'SpineShoulder', 'ElbowRight')

    hip_angle_left = calculate_angle(group, 'HipLeft', 'SpineBase', 'KneeLeft')
    knee_angle_left = calculate_angle(group, 'HipLeft', 'KneeLeft', 'AnkleLeft')
    ankle_angle_left = calculate_angle(group, 'KneeLeft', 'AnkleLeft', 'FootLeft')

    hip_angle_right = calculate_angle(group, 'HipRight', 'SpineBase', 'KneeRight')
    knee_angle_right = calculate_angle(group, 'HipRight', 'KneeRight', 'AnkleRight')
    ankle_angle_right = calculate_angle(group, 'KneeRight', 'AnkleRight', 'FootRight')
    
    
    # Extraer columnas adicionales
    subject_id = group['SubjectID'].iloc[0]
    gesture_label = group['GestureLabel'].iloc[0]
    repetition_number = group['RepetitionNumber'].iloc[0]
    correct_label = group['CorrectLabel'].iloc[0]
    position = group['Position'].iloc[0]
    
    # Almacenar la información en un diccionario
    angles.append({
        'SubjectID': subject_id,
        'GestureLabel': gesture_label,
        'RepetitionNumber': repetition_number,
        'CorrectLabel': correct_label,
        'Position': position,
        'ElbowAngleLeft': elbow_angle_left,
        'ElbowAngleRight': elbow_angle_right,
        'ShoulderAngleLeft': shoulder_angle_left,
        'ShoulderAngleRight': shoulder_angle_right,
        'WristAngleLeft': wrist_angle_left,
        'WristAngleRight': wrist_angle_right,
        'HipAngleLeft': hip_angle_left,
        'KneeAngleLeft': knee_angle_left,
        'AnkleAngleLeft': ankle_angle_left,
        'HipAngleRight': hip_angle_right,
        'KneeAngleRight': knee_angle_right,
        'AnkleAngleRight': ankle_angle_right,
        'LeftArmAngle': left_arm_angle,
        'RightArmAngle': right_arm_angle,
        'ArmsTogetherAngle': arms_together_angle
    })

In [17]:
# Crear un DataFrame a partir de la lista de diccionarios
angles_df = pd.DataFrame(angles)
angles_df

,SubjectID,GestureLabel,RepetitionNumber,CorrectLabel,Position,ElbowAngleLeft,ElbowAngleRight,ShoulderAngleLeft,ShoulderAngleRight,WristAngleLeft,WristAngleRight,HipAngleLeft,KneeAngleLeft,AnkleAngleLeft,HipAngleRight,KneeAngleRight,AnkleAngleRight,LeftArmAngle,RightArmAngle,ArmsTogetherAngle
0,101,0,1,1,stand,18.731844,12.815119,142.560727,140.857143,2.598764,6.344406,104.574768,3.743551,64.246261,101.521850,4.855627,60.940986,156.791112,159.877408,152.580270
1,101,0,1,1,stand,18.682011,13.214648,142.757760,140.967721,2.522847,5.905771,104.499195,3.821802,64.334729,101.590425,4.885456,61.134582,156.667813,160.035203,152.602527
2,101,0,1,1,stand,18.530184,14.175895,142.724795,140.886094,3.239981,6.639703,104.558119,3.832570,64.078901,101.492094,4.776421,59.972082,156.747131,160.251764,152.596513
3,101,0,1,1,stand,18.525441,15.675640,142.796223,140.996819,3.488477,10.172814,104.502499,3.642046,63.451166,101.422947,4.655747,59.366231,156.715615,160.589502,152.653362
4,101,0,1,1,stand,18.440644,18.672491,142.755019,140.986030,3.640028,9.700975,104.554046,3.670732,63.212740,101.311001,4.601472,59.328043,156.793484,160.668306,152.675640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228296,307,8,9,1,stand,19.235168,16.366695,142.466122,142.949282,10.597409,8.590306,107.710008,17.152761,57.808079,94.948037,1.709497,41.616584,156.103380,158.651430,149.864559
228297,307,8,9,1,stand,19.337504,16.460773,142.614506,142.959204,9.097187,8.083043,107.352914,16.479229,57.733793,94.653231,1.263963,41.879291,156.056261,158.714378,149.734213
228298,307,8,9,1,stand,19.762179,16.292120,142.579738,142.903745,9.288112,10.157666,107.398951,16.368690,58.670153,94.346427,1.041109,78.704138,156.130164,158.836560,149.712017
228299,307,8,9,1,stand,20.339779,16.195526,142.685324,142.897308,7.335038,9.108455,107.436579,16.163032,57.798046,94.144821,1.015447,78.752737,156.114139,158.911807,149.496620


In [18]:
# guardar los datos en formato csv
angles_df.to_csv('../Resulados/angles.csv', index=False)